In [ ]:
from gym.envs.registration import register
from pathlib import Path
import pandapower as pp
from citylearn import GridLearn
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3 import SAC
import gym

grid = GridLearn() # to do later: pull grid out of the initial conditions, make external set_grid method

climate_zone = 1
data_path = Path("../citylearn/data/Climate_Zone_"+str(climate_zone))
buildings_states_actions = '../citylearn/buildings_state_action_space.json'

config = {
    "data_path":data_path,
    "climate_zone":climate_zone,
    "buildings_states_actions_file":buildings_states_actions,
    "hourly_timesteps":2,
    "grid":grid,
}

register(
    id='buildings-v2',
    entry_point='citylearn.energy_models:Building',
    max_episode_steps=1000,
    reward_threshold=0.0,
    kwargs=config
)

In [ ]:
models = []
envs = []
for i in range(32):
    env = gym.make("buildings-v2")
    envs += [env]
    models += [SAC(MlpPolicy, env, verbose=0, learning_rate=0.01, gamma=0.99, tau=3e-4, batch_size=64, learning_starts=4)]

In [ ]:
for i in range(5):
    for m in range(len(models)):
        models[m].learn(1)

In [ ]:
models[4].actor.parameters_to_vector()

In [ ]:
# check what the building loads are
env.grid.net.load

In [ ]:
# iterate through 1 timestep
for _ in range(1):
    for env in envs:
        action = [0,0,0,0]
        obs, rewards, dones, info = env.step(action)

In [ ]:
# check that the building loads have changed
env.grid.net.load